In [11]:
# !pip install pymupdf

In [ ]:
from openai import OpenAI
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import os
import pandas as pd
import time
import fitz  # PyMuPDF
import tiktoken  # Import tiktoken library
import pickle

api_dev_atwin="" #secret
os.environ["OPENAI_API_KEY"] = api_dev_atwin

In [ ]:

class DocumentSearch:
    def __init__(self, api_key, timer_enabled=True,embedding_model = "text-embedding-ada-002"):
        self.client = OpenAI(api_key=api_key)
        self.documents = []
        self.document_embeddings = []
        self.chunked_docs=[]
        self.embedding_model = embedding_model
        self.tokenizer = tiktoken.get_encoding('cl100k_base')  # Initialize the tokenizer for the embedding model
    def preprocess_documents(self, documents):
        # Perform text cleaning, tokenization, and normalization
        # preprocessed_docs = [preprocess_text(doc) for doc in documents]
        return documents

    def generate_embeddings(self, documents):
        embeddings = []
        for doc in documents:
            response = self.client.embeddings.create(
                input=doc,
                model=self.embedding_model
            ).data[0].embedding
            embeddings.append(response)
        return embeddings
    
    def chunk_documents(self, documents, max_tokens=8192):
        # chunked_docs=[]
        for doc in documents:
            tokens = self.tokenizer.encode(doc)
            for i in range(0, len(tokens), max_tokens):
                chunked_tokens = tokens[i:i + max_tokens]
                chunked_text = self.tokenizer.decode(chunked_tokens)
                self.chunked_docs.append(chunked_text)
        # return chunked_docs
    
    def load_documents(self, folder_path):
        # self.documents = [
        #     open(os.path.join(folder_path, file), 'r').read()
        #     for file in os.listdir(folder_path) if file.endswith(".txt")
        # ]
        # preprocessed_docs = self.preprocess_documents(self.documents)
        # self.chunk_documents(preprocessed_docs)
        # self.save_chunks('chunk_1')
        # self.load_chunks('chunk_1')
        
        # self.document_embeddings = self.generate_embeddings(self.chunked_docs)
        # self.save_embed('chunk_1')
        self.load_embed('chunk_1')
        
    def save_embed(self,chunk_name):
        with open(f"{chunk_name}.pkl", "wb") as file:
            # Dump the list into the file
            pickle.dump(self.document_embeddings , file)
    def load_embed(self, chunk_name):
        with open(f"{chunk_name}.pkl", "rb") as file:
            # Load the object from the file
            self.document_embeddings = pickle.load(file)
            
    def search_documents(self, user_prompt, top_k=3,):
        
        query_embedding = self.client.embeddings.create(
            input=user_prompt,
            model=self.embedding_model
        ).data[0].embedding
        
        similarities = cosine_similarity([query_embedding], self.document_embeddings)[0]
        top_indices = similarities.argsort()[-top_k:][::-1]
        top_documents = [self.chunked_docs[i] for i in top_indices]
        top_similarities = similarities[top_indices]
        return top_documents, top_similarities


In [77]:
#convert docs to txts

def convert_pdf_to_txt(pdf_path, txt_path):
    # Open the PDF file
    pdf_document = fitz.open(pdf_path)
    
    # Create a text file to write the content
    with open(txt_path, 'w', encoding='utf-8') as txt_file:
        # Iterate over each page in the PDF
        for page_num in range(len(pdf_document)):
            page = pdf_document.load_page(page_num)  # Load the page
            text = page.get_text()  # Extract the text
            txt_file.write(text)  # Write the text to the text file
            txt_file.write('\n\n')  # Add some space between pages

    print(f"Converted {pdf_path} to {txt_path}")

# Example usage
pdf_directory = 'dokumen_vektor'  # Directory containing PDF files
txt_directory = 'dokumen_vektor/converted_texts'  # Directory to save TXT files

# Create the output directory if it doesn't exist
os.makedirs(txt_directory, exist_ok=True)

# Convert all PDF files in the specified directory
for pdf_file in os.listdir(pdf_directory):
    if pdf_file.endswith('.pdf'):
        pdf_path = os.path.join(pdf_directory, pdf_file)
        txt_path = os.path.join(txt_directory, pdf_file.replace('.pdf', '.txt'))
        convert_pdf_to_txt(pdf_path, txt_path)


Converted dokumen_vektor\NRA PPATK terkait Money Laundering.pdf to dokumen_vektor/converted_texts\NRA PPATK terkait Money Laundering.txt
Converted dokumen_vektor\POJK 8-2023 - APU PPT dan PPPSPM di SJK.pdf to dokumen_vektor/converted_texts\POJK 8-2023 - APU PPT dan PPPSPM di SJK.txt


In [80]:

# run search
api_key = api_dev_atwin
search_engine_3large = DocumentSearch(api_key, timer_enabled=True)
folder_material = r'dokumen_vektor/converted_texts'
search_engine_3large.load_documents(folder_material)
# search_engine_3large.load_chunks

In [81]:
query = " steps in facing risks in money laundering  during the latest NRA period"
top_documents, top_similarities = search_engine_3large.search_documents(query,top_k=5,)

# for doc, sim in zip(top_documents, top_similarities):
#    print(f"Similarity: {sim}")
#    print(f"Document: {doc[:130]}...")

#docs prep
top_documents=top_documents.replace('\n \n','\n')

ValueError: Input contains NaN.